# IBM  CAPSTONE PROJECT(The battle of neighborhoods)

## LOCATION RECOMMENDOR SYSTEM



#### AGENDA:

1.Building a dataframe of the hotspots(containment zones) in Kerala,India.

2.Getting latitude and longitude co ordinates and then visualization.

3.Using foursquare API to extract hospital venues near the hotspots.

4.Performing KMeans Clustering algorithm.

5.Finding the best location for setting up the medical rehabilitation centre along with the insights taken from EDA.

### IMPORTING LIBRARIES

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
!pip install geopy
!pip install geocoder
from geopy.geocoders import Nominatim
import geocoder

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:011


### LOADING THE DATASET

In [3]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_50898805c015402da81c9cd63f2bc52e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jLjbFCuN30a1-7N-tahU4utUzTCXQ3bXoqnsTDDkX7H1',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_50898805c015402da81c9cd63f2bc52e.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-0osqkjjjr56iyh',Key='kerala containment zones.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Sl. No.,District,LSGs needing special attention,Containment Zones (Ward)
0,1,Kannur,Aalakkod,1
1,2,Kannur,Chapparappadavu,1
2,3,Kannur,Chirakkal,2
3,4,Kannur,Chittaripparamba,2
4,5,Kannur,Irikkoor,1


### DATA CLEANING

In [4]:
df_data_1=df_data_1.drop(["Sl. No."],axis=1)

In [5]:
df_data_1 =df_data_1.rename(columns={"LSGs needing special attention": "Hotspot"})

In [6]:
df_data_1.head()

,District,Hotspot,Containment Zones (Ward)
0,Kannur,Aalakkod,1
1,Kannur,Chapparappadavu,1
2,Kannur,Chirakkal,2
3,Kannur,Chittaripparamba,2
4,Kannur,Irikkoor,1


### GETTING LATITUDE AND LONGITUDE CO ORDINATES

In [7]:
def get_latlng(Hotspot):
    
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kerala,India'.format(Hotspot))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:
coords = [ get_latlng(Hotspot) for Hotspot in df_data_1["Hotspot"].tolist() ]

coords

[[11.23052000000007, 75.91652000000005],
 [12.127200000000073, 75.41569000000004],
 [11.911070000000052, 75.36464000000007],
 [12.050000000000068, 75.35000000000008],
 [11.984210000000076, 75.55319000000003],
 [11.992710000000045, 75.68032000000005],
 [12.080930000000023, 75.26097000000004],
 [11.874720000000025, 75.37518000000006],
 [10.129210000000057, 76.29474000000005],
 [11.759340000000066, 75.57878000000005],
 [11.971770000000049, 75.41338000000007],
 [11.830610000000036, 75.56666000000007],
 [11.819440000000043, 75.56024000000008],
 [12.03788000000003, 75.24662000000006],
 [11.261460000000056, 75.95392000000004],
 [11.931630000000041, 75.57214000000005],
 [12.019400000000076, 75.24595000000005],
 [11.669750000000022, 75.72609000000006],
 [11.929890000000057, 75.43755000000004],
 [11.797720000000027, 75.45038000000005],
 [12.001520000000028, 75.68708000000004],
 [11.837060000000065, 75.44770000000005],
 [11.941260000000057, 75.35319000000004],
 [12.105920000000026, 75.20590000000

In [9]:
data_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_data_1['Latitude'] = data_coords['Latitude']
df_data_1['Longitude'] = data_coords['Longitude']
print(df_data_1.shape)
print(df_data_1)

(128, 5)
               District                   Hotspot  Containment Zones (Ward)  \
0                Kannur                  Aalakkod                         1   
1                Kannur           Chapparappadavu                         1   
2                Kannur                 Chirakkal                         2   
3                Kannur          Chittaripparamba                         2   
4                Kannur                  Irikkoor                         1   
5                Kannur                Iritty (M)                         1   
6                Kannur  Kadannappalli-Panappuzha                         2   
7                Kannur                Kannur (C)                         6   
8                Kannur               Karumathoor                         2   
9                Kannur                    Panoor                         3   
10               Kannur                Kolacherri                         1   
11               Kannur         Koothuparam

In [10]:
df_data_1.head()

,District,Hotspot,Containment Zones (Ward),Latitude,Longitude
0,Kannur,Aalakkod,1,11.23052,75.91652
1,Kannur,Chapparappadavu,1,12.12720,75.41569
2,Kannur,Chirakkal,2,11.91107,75.36464
3,Kannur,Chittaripparamba,2,12.05000,75.35000
4,Kannur,Irikkoor,1,11.98421,75.55319


In [11]:
df_data_1=df_data_1.drop(["Containment Zones (Ward)"],axis=1)

In [12]:
df_data_1.isnull().sum()

District     0
Hotspot      0
Latitude     0
Longitude    0
dtype: int64

In [13]:
df_data_1.to_csv("df_data_1.csv", index=False)

### VISUALIZATION

In [14]:
address = 'Kerala, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kerala, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kerala, India are 10.3528744, 76.5120396.


In [16]:
!pip install folium
import folium
kerala_hotspots = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, City in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['Hotspot']):
    label = '{}'.format('Hotspot')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(kerala_hotspots)  
    
kerala_hotspots

     |████████████████████████████████| 102kB 7.6MB/s ta 0:00:011


In [17]:
kerala_hotspots.save('kerala_hotspots.html')

### USING FOURSQUARE API

In [18]:
CLIENT_ID = 'M32ME4PM5DD02M3GPYZZDO1GYM5P0NNNIUYL4DZDFYGS43AC'
CLIENT_SECRET = 'KT3GOSYC1QUPL5ZPHSJ1W2DOUAOKAVLINTXXX2ERWZ2ROKXU' 
VERSION = '20200703' 
Category_ID='4bf58dd8d48988d196941735'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('CATEGORY_ID:' + Category_ID)

Your credentails:
CLIENT_ID: M32ME4PM5DD02M3GPYZZDO1GYM5P0NNNIUYL4DZDFYGS43AC
CLIENT_SECRET:KT3GOSYC1QUPL5ZPHSJ1W2DOUAOKAVLINTXXX2ERWZ2ROKXU
CATEGORY_ID:4bf58dd8d48988d196941735


In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, Hotspot in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['Hotspot']):
    
    # creating the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=Hospital".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # making the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    
    for venue in results:
        venues.append((
            Hotspot,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
venues_data = pd.DataFrame(venues)
venues_data.columns = ['Hotspot', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_data.shape)
venues_data.head()

(111, 7)


,Hotspot,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Chittaripparamba,12.05000,75.35000,lourde hospital,12.041304,75.355864,Hospital
1,Chittaripparamba,12.05000,75.35000,Nayanar Memorial Hospital,12.037710,75.363128,Hospital
2,Iritty (M),11.99271,75.68032,Amala Hospital,11.981325,75.672449,Hospital
3,Iritty (M),11.99271,75.68032,Amala Speciality Hospital,11.982065,75.666480,Hospital
4,Kadannappalli-Panappuzha,12.08093,75.26097,Cheruthazham Primary Health Centre,12.078582,75.265875,Hospital


In [21]:
venues_data.groupby(["Hotspot"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Hotspot,,,,,,
Ajanur,3,3,3,3,3,3
Amballoor,2,2,2,2,2,2
Balaramapuram,2,2,2,2,2,2
Chalakkudy(M),2,2,2,2,2,2
Chemnad,3,3,3,3,3,3
Chittaripparamba,2,2,2,2,2,2
Iritty (M),2,2,2,2,2,2
Kadannappalli-Panappuzha,2,2,2,2,2,2
Kanhangad (M),4,4,4,4,4,4


In [22]:
venues_data['VenueCategory'].unique()[:50]

array(['Hospital', 'Hospital Ward', 'Medical Center'], dtype=object)

In [23]:
kh_onehot = pd.get_dummies(venues_data[['VenueCategory']], prefix="", prefix_sep="")


kh_onehot['Hotspot'] = venues_data['Hotspot'] 


fixed_columns = [kh_onehot.columns[-1]] + list(kh_onehot.columns[:-1])
kh_onehot = kh_onehot[fixed_columns]

print(kh_onehot.shape)
kh_onehot.head()

(111, 4)


,Hotspot,Hospital,Hospital Ward,Medical Center
0,Chittaripparamba,1,0,0
1,Chittaripparamba,1,0,0
2,Iritty (M),1,0,0
3,Iritty (M),1,0,0
4,Kadannappalli-Panappuzha,1,0,0


In [24]:
kh_onehot.tail()

,Hotspot,Hospital,Hospital Ward,Medical Center
106,Neyyattinkara (M),1,0,0
107,Kattappana (M),1,0,0
108,Olavanna,1,0,0
109,Thirunelli,1,0,0
110,Thirunelli,0,0,1


In [25]:
len(kh_onehot[kh_onehot["Hospital"] > 0])

109

In [26]:
KH= kh_onehot[["Hotspot","Hospital"]]

KH.head()

,Hotspot,Hospital
0,Chittaripparamba,1
1,Chittaripparamba,1
2,Iritty (M),1
3,Iritty (M),1
4,Kadannappalli-Panappuzha,1


In [27]:
hospitals_per_hotspots=KH['Hospital'].groupby(KH['Hotspot'])

print(hospitals_per_hotspots.sum())

Hotspot
Ajanur                       3
Amballoor                    2
Balaramapuram                2
Chalakkudy(M)                2
Chemnad                      3
Chittaripparamba             2
Iritty (M)                   2
Kadannappalli-Panappuzha     2
Kanhangad (M)                4
Kanjoor                      1
Kannur (C)                   6
Karukachal                   2
Kattappana (M)               1
Kochi (C)                   10
Kottayam (M)                 4
Kuzhalmannam                 1
Mannarkad (M)                1
Neyyattinkara (M)            1
Olavanna                     1
Pallikkathodu                2
Payyannur (M)               12
Perumatti                    8
Pirayiri                     1
Sreekandapuram (M)           1
Sreemoolanagaram             1
Thachampara                  1
Thalasheery (M)              4
Thirunelli                   1
Thiruvananthapuram (C)       8
Thrikodithanam               1
Thrissur (C)                 6
Trikaripur                   1


### KMeans ALGORITHM

In [28]:
from sklearn.cluster import KMeans
k_clusters=3
kh_clustering=KH.drop(['Hotspot'],1)
kmeans = KMeans(n_clusters=k_clusters, random_state=2).fit(kh_clustering)
kmeans.labels_[1:10]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
kh_merged = KH.copy()
kh_merged["Cluster Labels"] =kmeans.labels_

In [30]:
kh_merged.rename(columns={"Hotspot": "Hotspot"}, inplace=True)
kh_merged.head()

,Hotspot,Hospital,Cluster Labels
0,Chittaripparamba,1,0
1,Chittaripparamba,1,0
2,Iritty (M),1,0
3,Iritty (M),1,0
4,Kadannappalli-Panappuzha,1,0


In [31]:
kh_merged.tail()

,Hotspot,Hospital,Cluster Labels
106,Neyyattinkara (M),1,0
107,Kattappana (M),1,0
108,Olavanna,1,0
109,Thirunelli,1,0
110,Thirunelli,0,1


In [32]:
kh_merged= kh_merged.join(venues_data[['VenueLatitude','VenueLongitude']])

kh_merged.head()

,Hotspot,Hospital,Cluster Labels,VenueLatitude,VenueLongitude
0,Chittaripparamba,1,0,12.041304,75.355864
1,Chittaripparamba,1,0,12.037710,75.363128
2,Iritty (M),1,0,11.981325,75.672449
3,Iritty (M),1,0,11.982065,75.666480
4,Kadannappalli-Panappuzha,1,0,12.078582,75.265875


In [33]:
print(kh_merged.shape)
kh_merged.sort_values(["Cluster Labels"], inplace=True)
kh_merged

(111, 5)


,Hotspot,Hospital,Cluster Labels,VenueLatitude,VenueLongitude
0,Chittaripparamba,1,0,12.041304,75.355864
80,Thrissur (C),1,0,10.512808,76.214020
79,Thrissur (C),1,0,10.510587,76.210626
78,Kanjoor,1,0,10.137020,76.419217
76,Kochi (C),1,0,9.950778,76.258381
75,Kochi (C),1,0,9.924614,76.252644
74,Kochi (C),1,0,9.947920,76.255818
73,Kochi (C),1,0,9.947109,76.256061
72,Kochi (C),1,0,9.927002,76.255886
71,Kochi (C),1,0,9.944106,76.259693


In [34]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [35]:
hospital_map = folium.Map(location=[latitude,longitude], zoom_start=12)

X = kh_merged['VenueLatitude']
Y = kh_merged['VenueLongitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=3, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red','green','blue']
kh_merged['Cluster Labels'] = clusters

for latitude, longitude, borough, cluster in zip(kh_merged['VenueLatitude'], kh_merged['VenueLongitude'], kh_merged['Hospital'], kh_merged['Cluster Labels']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(hospital_map)  

hospital_map

In [37]:
#kh_merged.loc[kh_merged['Cluster Labels'] == 0].shape
kh_merged.loc[kh_merged['Cluster Labels'] == 0]

,Hotspot,Hospital,Cluster Labels,VenueLatitude,VenueLongitude
0,Chittaripparamba,1,0,12.041304,75.355864
108,Olavanna,1,0,11.226727,75.833084
24,Sreekandapuram (M),1,0,12.042302,75.493713
23,Payyannur (M),1,0,12.119970,75.209140
22,Payyannur (M),1,0,12.107931,75.213179
21,Payyannur (M),1,0,12.107814,75.213774
20,Payyannur (M),1,0,12.108342,75.210686
19,Payyannur (M),1,0,12.110314,75.208100
18,Payyannur (M),1,0,12.109295,75.208412
17,Payyannur (M),1,0,12.106005,75.209613


In [39]:
#kh_merged.loc[kh_merged['Cluster Labels'] == 1].shape
kh_merged.loc[kh_merged['Cluster Labels'] == 1]

,Hotspot,Hospital,Cluster Labels,VenueLatitude,VenueLongitude
65,Amballoor,1,1,8.569962,76.872314
64,Amballoor,1,1,8.574775,76.868768
106,Neyyattinkara (M),1,1,8.402403,77.085920
105,Vanchiyoor,1,1,8.510163,76.942614
104,Vanchiyoor,1,1,8.486949,76.927229
103,Vanchiyoor,1,1,8.502028,76.950907
102,Vanchiyoor,1,1,8.508599,76.937960
101,Vanchiyoor,1,1,8.484896,76.941842
100,Vanchiyoor,1,1,8.499657,76.944386
99,Vanchiyoor,1,1,8.499422,76.943018


In [41]:
#kh_merged.loc[kh_merged['Cluster Labels'] == 2].shape
kh_merged.loc[kh_merged['Cluster Labels'] == 2]

,Hotspot,Hospital,Cluster Labels,VenueLatitude,VenueLongitude
80,Thrissur (C),1,2,10.512808,76.214020
79,Thrissur (C),1,2,10.510587,76.210626
78,Kanjoor,1,2,10.137020,76.419217
76,Kochi (C),1,2,9.950778,76.258381
75,Kochi (C),1,2,9.924614,76.252644
74,Kochi (C),1,2,9.947920,76.255818
73,Kochi (C),1,2,9.947109,76.256061
72,Kochi (C),1,2,9.927002,76.255886
71,Kochi (C),1,2,9.944106,76.259693
70,Kochi (C),1,2,9.940944,76.255432


## OBSERVATIONS

After performing the KMeans clustering algorithm,we were able to categorize the hotspots including containment zones into 3 clusters based on the frequency of occurrence of a hospital nearby
•	Cluster 0 : highest number of hospitals in the neighborhoods
•	Cluster 1 : lowest number of hospitals in the neighborhoods
•	Cluster 2 : moderate number of hospitals in the neighborhoods
in red, green and blue respectively.
Hence neighborhoods in cluster 0 would be the best cluster to set up the medical rehabilitation center. Since we are concentrating on the Covid pandemic we need to incorporate the EDA done with other datasets regarding the patient count,recovery ststus and the phsyco social support data


While exploring other datasets in our project we found that Malappuram has the higher concentration of patients highly in need of a recovery , since it topped both in count of hotspots and count of mental support needed and the hospitals identified in the hotspots from  this district happens to be in cluster 0. Incorporating all these insights ,this location will be a boon for the establishment of a Rehabilitation centre.We will try to get the hotspots in malappuram district on map.

In [42]:
sol_data=df_data_1.iloc[[111,112,113,114]]

In [43]:
sol_data.head()

,District,Hotspot,Latitude,Longitude
111,Malappuram,Parappanangadi (M),11.05000,75.86667
112,Malappuram,Pulpatta,11.15825,76.06111
113,Malappuram,Ponnani Taluk,10.76695,75.92523
114,Malappuram,Tanur (M),10.96667,75.86667


In [44]:
!pip install folium
import folium
Malappuram_hotspots = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, City in zip(sol_data['Latitude'], sol_data['Longitude'], sol_data['Hotspot']):
    label = '{}'.format('Hotspot')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Malappuram_hotspots)  
    
Malappuram_hotspots

These are the hotspots identified in Malappuram district. The four locations spotted here are the LSG's(local self government) needing special attention. A total of 94 subwards here are fighting to contain the covid pandemic(found through bar chart). Since the ward locations were difficult to fetch on a map we can come to a conclusion that the rehab centre can be set up surrounding these hotspots for maximum benefit.